In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        continue

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
import requests
from PIL import Image
from io import BytesIO
import torch
from fastai.vision.all import *
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import torchvision.transforms as transforms
from tqdm import tqdm
from glob import glob
import gc
import joblib

In [22]:
# Function to open an image from a file path
def open_image_from_path(image_path: str):
    img = Image.open(image_path).convert('RGB')
    return img

In [23]:
# Set up transformation (this should match the transformation used during training)
def preprocess_image(image):
    transformation_pipeline = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transformation_pipeline(image)
    return img_tensor.unsqueeze(0)

In [26]:
def predict_image(image_url):
    img = open_image_from_path(image_url)
    img_tensor = preprocess_image(img)

    dls = ImageDataLoaders.from_dsets([img_tensor], [[0]])  # This needs adjustment for multi-label
    dls.c = 14  # Correct for 14 possible disease classes

    # Load your pre-trained FastAI model (example)
    learner = vision_learner(dls, arch='coatnet_2_rw_224.sw_in12k_ft_in1k', pretrained=False,
                             loss_func=nn.BCEWithLogitsLoss())
    learner.load('/kaggle/input/coatnet-stage2-0139/models/model')
    
    learner.model = torch.nn.DataParallel(learner.model)
    learner.to('cuda')
    
    learner.model.eval()
    
    with torch.no_grad():
        logits = learner.model(img_tensor.cuda())  # Ensure tensor is on GPU
        probabilities = torch.sigmoid(logits)  # Convert logits to probabilities

    return probabilities

In [27]:
# Example usage:
image_url = '/kaggle/input/data/images_012/images/00028173_005.png'
prediction = predict_image(image_url)
print(f"Prediction: {prediction}")

Prediction: tensor([[1.8067e-02, 2.4231e-02, 2.8272e-01, 9.1223e-03, 1.7705e-02, 1.2648e-02,
         8.9783e-04, 4.8572e-02, 1.7295e-02, 3.0457e-03, 4.9020e-02, 1.4813e-02,
         1.0750e-03, 3.6869e-05]], device='cuda:0')


In [ ]:
labels_df['Finding_Labels'] = labels_df['Finding_Labels'].apply(lambda s: [l for l in str(s).split('|')])

num_glob = glob('/kaggle/input/data/*/images/*.png')
img_path = {os.path.basename(x): x for x in num_glob}

labels_df['Paths'] = labels_df['Image_Index'].map(img_path.get)
labels_df.head()

In [ ]:
unique_patients = np.unique(labels_df['Patient_ID'])
len(unique_patients)

In [ ]:
from sklearn.model_selection import train_test_split

# train-70
# val-10
# test-20
train_val_df_patients, test_df_patients = train_test_split(unique_patients, 
                                   test_size = 0.2,
                                   random_state = SEED,
                                    shuffle= True
                                   )
len(train_val_df_patients)

In [ ]:
train_val_df = labels_df[labels_df['Patient_ID'].isin(train_val_df_patients)]
test_df = labels_df[labels_df['Patient_ID'].isin(test_df_patients)]

# test_df=test_df.head(1568)

print('test size', test_df.shape[0])
print('test size', labels_df.shape[0] - train_val_df.shape[0])

In [ ]:
convnextv2_result= get_roc_auc('convnextv2_large.fcmae_ft_in22k_in1k','/kaggle/input/convnextv2-stage2/models/model')
preds = convnextv2_result['preds']
torch.save(preds, 'convnextv2_result.pt')

In [ ]:
swinv2_result= get_roc_auc('swinv2_cr_small_ns_224.sw_in1k', '/kaggle/input/swin-full-nofinding/models/model')
preds = swinv2_result['preds']
torch.save(preds, 'swinv2_result.pt')

In [ ]:
maxvit_result= get_roc_auc('maxxvitv2_rmlp_base_rw_224.sw_in12k_ft_in1k','/kaggle/input/maxvit-stage2/models/model')
preds = maxvit_result['preds']
torch.save(preds, 'maxvit_result.pt')

In [ ]:
volod2_result= get_roc_auc('volo_d2_224.sail_in1k', '/kaggle/input/volo-d2-70-10-20-split/models/model')
preds = volod2_result['preds']
torch.save(preds, 'volod2_result.pt')

In [ ]:
densenet121_result= get_roc_auc(models.densenet121,'/kaggle/input/densenet121-70-10-20-split/models/model')
preds = densenet121_result['preds']
torch.save(preds, 'densenet121_result.pt')

In [ ]:
# Example usage:
image_url = '/kaggle/input/data/images_012/images/00028173_003.png'
prediction = predict_image(image_url)
print(f"Prediction: {prediction}")

In [ ]:
from itertools import permutations,combinations

def generate_permutations(input_list):
    perms= permutations(input_list)
    return [i for i in perms]

def generate_combinations(input_list, n):
    combs= combinations(input_list, n)
    return [i for i in combs]

In [ ]:
model_pred_map={
    'coatnet': coatnet_result['preds'],
    'convnext': convnextv2_result['preds'],
    'maxvit': maxvit_result['preds'],
    'densenet121': densenet121_result['preds'],
    'volod2': volod2_result['preds'],
    'swinv2': swinv2_result['preds'],
#     'swinv2-xgboost': torch.from_numpy(swinv2_xgb_preds),
#     'swinv2-RF': swinv2_rf_preds
}

In [ ]:
model_name_pred_tuples=[]
for model_name in model_pred_map.keys():
    model_name_pred_tuples.append((model_name, model_pred_map[model_name]))
# list_tuples